In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


preprocessing the data

In [3]:
data=data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

In [4]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


encoding categorical data

In [5]:
#use when we have 2 categories
leg=LabelEncoder()
data['Gender']=leg.fit_transform(data['Gender'])

In [6]:
data.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
8230,758,Germany,1,32,4,162657.64,2,1,1,115525.13,0
2452,734,France,0,29,0,139994.66,1,1,0,17744.72,0
6037,711,Spain,1,32,5,0.00,2,1,1,147720.27,0
1019,633,France,1,31,1,0.00,1,1,0,48606.71,0
1506,747,France,1,27,10,0.00,2,0,0,13007.89,0


In [7]:
#One hot encoder when we have more than 2 categories
from sklearn.preprocessing import OneHotEncoder
oheGeo=OneHotEncoder()
geo_encoder=oheGeo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
oheGeo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [9]:
geo_encoded=pd.DataFrame(geo_encoder.toarray(),columns=oheGeo.get_feature_names_out(['Geography']))
geo_encoded

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


combining ohe columns iwth orignal data

In [10]:
data=pd.concat([data.drop(['Geography'],axis=1),geo_encoded],axis=1)

In [11]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


saving encoders and scaler

In [12]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(leg,file)

with open('one_hot_encoder_geography.pkl','wb') as file:
    pickle.dump(oheGeo,file)


dividing the dataset into train and test data

In [13]:
X=data.drop(['Exited'],axis=1)
Y=data['Exited']

In [14]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [15]:
ss=StandardScaler()
x_train=ss.fit_transform(x_train)
x_test=ss.fit_transform(x_test)


In [16]:
with open('scaler.pkl','wb') as file:
    pickle.dump(ss,file)

ANN Implementation

In [17]:
import sys
print(sys.version)
print(sys.executable)

3.12.0 | packaged by Anaconda, Inc. | (main, Oct  2 2023, 17:20:38) [MSC v.1916 64 bit (AMD64)]
d:\Generative AI\venv\python.exe


In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

ImportError: Traceback (most recent call last):
  File "C:\Users\peehu\AppData\Roaming\Python\Python312\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
x_train.shape

(8000, 12)

In [ ]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), #First hidden layer and is connected with input layer
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid') #Output layer for binary classification
])

C:\Users\peehu\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)

In [ ]:
#compiling the model
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#set up tensorboard: used to visualize the training process
log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [ ]:
#set up early stopping: used to stop training when a metrics stops improving
early_stopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)


In [ ]:
#training the model
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=50,callbacks=[tensorflow_callback,early_stopping])


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.8378 - loss: 0.3931 - val_accuracy: 0.8435 - val_loss: 0.3718
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8534 - loss: 0.3525 - val_accuracy: 0.8550 - val_loss: 0.3509
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.8593 - loss: 0.3515 - val_accuracy: 0.8605 - val_loss: 0.3385
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.8600 - loss: 0.3425 - val_accuracy: 0.8585 - val_loss: 0.3404
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.8618 - loss: 0.3415 - val_accuracy: 0.8550 - val_loss: 0.3486
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.8610 - loss: 0.3384 - val_accuracy: 0.8600 - val_loss: 0.3430
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.8634 - loss: 0.3354 - val_accuracy: 0.8655 - val_loss: 0.3372
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.8626 - loss: 0.3345 - val_ac

In [ ]:
model.save('model.h5')

In [ ]:
import sys
print(sys.executable)

d:\Generative AI\venv\python.exe


In [ ]:
import sys
!"{sys.executable}" -m pip install --upgrade tensorboard


In [ ]:
%reload_ext tensorboard

In [ ]:
#loading tensorboard
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [ ]:
%tensorboard --logdir logs/fit

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.